In [16]:
# pytorch
import torch
import torch.nn as nn
# Mobile Net
from MobileNetV2 import mobilenet_v2
# dataset
from torchvision import datasets
from torchvision import transforms
# dataloader
from torch.utils.data import DataLoader
# Util
import time
import numpy as np
import matplotlib.pyplot as plt
# tensorboard
from torch.utils.tensorboard import SummaryWriter

In [17]:
# 超参数
input_size = 224
batch_size = 64
n_worker = 0
lr = 0.001
epochs = 40

In [18]:
# 划分数据集函数
from typing import Optional, Tuple
import torch
from torch.utils.data import Dataset, random_split

def train_test_split(
    dataset: Dataset,
    test_ratio: float,
    seed: Optional[int] = None,
) -> Tuple[Dataset, Dataset]:
    """Splits a dataset into random train and test subsets.
    Args:
        dataset (Dataset): dataset.
        test_ratio (float): test proportion (between 0 and 1).
        seed (int, optional): seed. Defaults to None.
    Returns:
        Tuple[Dataset, Dataset]: train and test datasets.
    """
    # Define generator
    generator = torch.Generator()
    if seed is not None:
        generator.manual_seed(seed)

    # Define lengths of subsets
    train_ratio = 1 - test_ratio
    train_size = int(train_ratio * len(dataset))
    test_size = len(dataset) - train_size
    lengths = [train_size, test_size]

    # Split
    train_dataset, test_dataset = random_split(dataset, lengths, generator)

    return train_dataset, test_dataset

In [19]:
# 生成训练数据集
set_path = "image/train_image"
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
data_transform = transforms.Compose([
        transforms.RandomResizedCrop(input_size, scale=(0.2, 1.0)), 
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
])
dataset = datasets.ImageFolder(set_path, transform=data_transform)
train_dataset, val_dataset = train_test_split(dataset, 0.2, 2)

In [20]:
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=n_worker, pin_memory=True)
val_loader = DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, 
    num_workers=n_worker)

In [21]:
test_path = "image/test_image"
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
data_transform = transforms.Compose([
        transforms.RandomResizedCrop(input_size, scale=(0.2, 1.0)), 
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
])
test_dataset = datasets.ImageFolder(test_path, transform=data_transform)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, 
    num_workers=n_worker)

In [6]:
# model = mobilenet_v2()
save_path = 'save_model/ecg_id.pt'
model = torch.load(save_path)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
writer = SummaryWriter()

In [7]:
def train(epoch):
    model.train()
    train_loss = 0
    for data, label in train_loader:
        data, label = data.cuda(), label.cuda()
        start_train = time.time()
        # clear the grad
        optimizer.zero_grad()
        output = model(data)
        # loss function
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
    train_loss = train_loss / len(train_loader.dataset)
    writer.add_scalar("Loss/train", train_loss, epoch)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch, train_loss))

In [12]:
def val(epoch):       
    model.eval()
    val_loss = 0
    gt_labels = []
    pred_labels = []
    with torch.no_grad():
        for data, label in val_loader:
            data, label = data.cuda(), label.cuda()
            output = model(data)
            preds = torch.argmax(output, 1)
            gt_labels.append(label.cpu().data.numpy())
            pred_labels.append(preds.cpu().data.numpy())
            loss = criterion(output, label)
            val_loss += loss.item()*data.size(0)
    val_loss = val_loss/len(val_loader.dataset)
    writer.add_scalar("Loss/val", val_loss, epoch)
    gt_labels, pred_labels = np.concatenate(gt_labels), np.concatenate(pred_labels)
    acc = np.sum(gt_labels==pred_labels)/len(pred_labels)
    writer.add_scalar("Acc/train", acc, epoch)
    print('Epoch: {} \tValidation Loss: {:.6f}, Accuracy: {:6f}'.format(epoch, val_loss, acc))

In [13]:
def test(epoch):       
    model.eval()
    gt_labels = []
    pred_labels = []
    with torch.no_grad():
        for data, label in test_loader:
            data, label = data.cuda(), label.cuda()
            output = model(data)
            preds = torch.argmax(output, 1)
            gt_labels.append(label.cpu().data.numpy())
            pred_labels.append(preds.cpu().data.numpy())
    gt_labels, pred_labels = np.concatenate(gt_labels), np.concatenate(pred_labels)
    acc = np.sum(gt_labels==pred_labels)/len(pred_labels)
    print('Epoch: {} Accuracy: {:6f}'.format(epoch, acc))
    return acc

In [14]:
for epoch in range(1, epochs + 1):
    train(epoch)
    if test(epoch) > 0.9:
        break
writer.flush()
writer.close()

Epoch: 0 	Training Loss: 0.111609
Epoch: 0 Accuracy: 0.547222
Epoch: 1 	Training Loss: 0.094907
Epoch: 1 Accuracy: 0.560556
Epoch: 2 	Training Loss: 0.089110
Epoch: 2 Accuracy: 0.521944
Epoch: 3 	Training Loss: 0.122679
Epoch: 3 Accuracy: 0.463611
Epoch: 4 	Training Loss: 0.143145
Epoch: 4 Accuracy: 0.494444
Epoch: 5 	Training Loss: 0.161332
Epoch: 5 Accuracy: 0.564722
Epoch: 6 	Training Loss: 0.081993
Epoch: 6 Accuracy: 0.570278
Epoch: 7 	Training Loss: 0.070712
Epoch: 7 Accuracy: 0.578611
Epoch: 8 	Training Loss: 0.105623
Epoch: 8 Accuracy: 0.575278
Epoch: 9 	Training Loss: 0.093387
Epoch: 9 Accuracy: 0.583889
Epoch: 10 	Training Loss: 0.102049
Epoch: 10 Accuracy: 0.596111


KeyboardInterrupt: 